In [1]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

from matplotlib import pyplot as plt

from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

In [2]:
# ==========
# helping methods
# ==========

# generate a list from lower and upper bound
def gen_list(p0, pn, delta, dig=5):
    ret = []
    i = p0
    while i < pn:
        ret.append(float(i))
        i += delta
        i = round(i, dig)
    return ret

# padding and zero padding
def padding(origin, a_list, b_list):
    return np.hstack((a_list, origin, b_list))

def zero_padding(origin, num):
    zero_list = [0 for i in range(num)]
    return padding(origin, zero_list, zero_list)

# trainning pairs
def gen_pair(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-1-length)
        train = current_t[p-length:p+length+1]
        solu = next_t[p]
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

# trainning pairs (in average)
def gen_pair_ave(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-2-length)
        train0 = current_t[p-length-1:p+length+2]
        solu1 = next_t[p]
        solu2 = next_t[p+1]
        train = []
        for j in range(len(train0)-1):
                train.append(0.5 * (train0[j] + train0[j+1]))
        solu = 0.5 * (solu1 + solu2)
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

In [3]:
# the analytical representation of exact solution
def heat_equ_analytical_solu(x, t):
    return np.sin(np.pi * x) * np.exp(-np.power(np.pi, 2) * t)

In [4]:
# restnet
class ResNet(nn.Module):
    def __init__(self, i, h, o):
        super(ResNet, self).__init__()
        self.linear1 = nn.Linear(i, h)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(h, o)
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu1(out)
        out = self.linear2(out)
        out = self.relu2(out)
        return out + torch.mean(x)

    def load_model(self, save_path):
        self.load_state_dict(torch.load(save_path))

    def save_model(self, save_path):
        torch.save(self.state_dict(), save_path)

In [5]:
def calc_next_time(current, model, length=3):
    ret = []
    p = np.hstack(([0 for i in range(length)], current, [0 for i in range(length)])).tolist()
    for index in range(len(current)):
        i = index + 3
        seg = p[i-length:i+length+1]
        tensor_seg = torch.FloatTensor(seg)
        out = model(tensor_seg)
        ret.append(out.item())
    return ret

## $\Delta x=\frac{1}{20}$ $\Delta t=\frac{1}{40}$

In [6]:
x_1 = arange(0, 2 * np.pi, 1/20)
t_1 = arange(0, 2 * np.pi, 1/40)
X_1,T_1 = meshgrid(x_1, t_1) # grid of point
Z_1 = heat_equ_analytical_solu(X_1, T_1) # evaluation of the function on the grid

In [7]:
padding_1 = []
for z in Z_1:
    p = np.hstack(([0,0,0], z, [0,0,0]))
    padding_1.append(p.tolist())

In [8]:
model_1 = ResNet(7, 6, 1)
optimizer_1 = optim.Adam(model_1.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion_1 = nn.MSELoss()
model_1.train()

ResNet(
  (linear1): Linear(in_features=7, out_features=6, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (relu2): ReLU()
)

In [9]:
model_1.load_model("model_xx delta x=1 20 delta t=1 40")

In [10]:
prediction_1 = []
prediction_1.append(torch.FloatTensor(Z_1[0]).tolist())
for i in range(len(Z_1)-1):
    prediction_1.append(calc_next_time(Z_1[i], model_1))
    print("time", i, "is done", '", an example is:"', prediction_1[i+1][0])

time 0 is done ", an example is:" 0.1313488483428955
time 1 is done ", an example is:" 0.10262860357761383
time 2 is done ", an example is:" 0.08018821477890015
time 3 is done ", an example is:" 0.06265456229448318
time 4 is done ", an example is:" 0.04895474761724472
time 5 is done ", an example is:" 0.03825048357248306
time 6 is done ", an example is:" 0.029886778444051743
time 7 is done ", an example is:" 0.023351846262812614
time 8 is done ", an example is:" 0.018245818093419075
time 9 is done ", an example is:" 0.014256254769861698
time 10 is done ", an example is:" 0.01113903522491455
time 11 is done ", an example is:" 0.008703416213393211
time 12 is done ", an example is:" 0.006800359580665827
time 13 is done ", an example is:" 0.005313418339937925
time 14 is done ", an example is:" 0.004151606000959873
time 15 is done ", an example is:" 0.003243831219151616
time 16 is done ", an example is:" 0.0025345473550260067
time 17 is done ", an example is:" 0.001980352681130171
time 18 i

time 143 is done ", an example is:" 6.235210695063878e-17
time 144 is done ", an example is:" 4.871842675903362e-17
time 145 is done ", an example is:" 3.806583601189812e-17
time 146 is done ", an example is:" 2.974250315165069e-17
time 147 is done ", an example is:" 2.3239118185891404e-17
time 148 is done ", an example is:" 1.815773941117203e-17
time 149 is done ", an example is:" 1.4187434598317557e-17
time 150 is done ", an example is:" 1.1085264110143613e-17
time 151 is done ", an example is:" 8.661401576597852e-18
time 152 is done ", an example is:" 6.767531859488293e-18
time 153 is done ", an example is:" 5.2877686093574805e-18
time 154 is done ", an example is:" 4.131564914042604e-18
time 155 is done ", an example is:" 3.228172015752003e-18
time 156 is done ", an example is:" 2.5223121224857498e-18
time 157 is done ", an example is:" 1.9707927577884688e-18
time 158 is done ", an example is:" 1.5398665036316452e-18
time 159 is done ", an example is:" 1.203165116833773e-18
time 16

In [11]:
diff_1 = (prediction_1-Z_1).tolist()

In [12]:
import csv

with open("prediction_12 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(prediction_1)

In [13]:
import csv

with open("diff_12 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(diff_1)

## $\Delta x=\frac{1}{40}$ $\Delta t=\frac{1}{40}$

In [14]:
x_2 = arange(0, 2 * np.pi, 1/40)
t_2 = arange(0, 2 * np.pi, 1/40)
X_2,T_2 = meshgrid(x_2, t_2) # grid of point
Z_2 = heat_equ_analytical_solu(X_2, T_2) # evaluation of the function on the grid

In [15]:
padding_2 = []
for z in Z_2:
    p = np.hstack(([0,0,0], z, [0,0,0]))
    padding_2.append(p.tolist())

In [16]:
model_2 = ResNet(7, 6, 1)
optimizer_2 = optim.Adam(model_2.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion_2 = nn.MSELoss()
model_2.train()

ResNet(
  (linear1): Linear(in_features=7, out_features=6, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (relu2): ReLU()
)

In [17]:
model_2.load_model("model_xx delta x=1 40 delta t=1 40")

In [18]:
prediction_2 = []
prediction_2.append(torch.FloatTensor(Z_2[0]).tolist())
for i in range(len(Z_2)-1):
    prediction_2.append(calc_next_time(Z_2[i], model_2))
    print("time", i, "is done", '", an example is:"', prediction_2[i+1][0])

time 0 is done ", an example is:" 0.06690555810928345
time 1 is done ", an example is:" 0.05227624252438545
time 2 is done ", an example is:" 0.04084571450948715
time 3 is done ", an example is:" 0.03191453963518143
time 4 is done ", an example is:" 0.02493622712790966
time 5 is done ", an example is:" 0.019483763724565506
time 6 is done ", an example is:" 0.015223517082631588
time 7 is done ", an example is:" 0.011894799768924713
time 8 is done ", an example is:" 0.009293926879763603
time 9 is done ", an example is:" 0.007261752150952816
time 10 is done ", an example is:" 0.005673923995345831
time 11 is done ", an example is:" 0.004433284979313612
time 12 is done ", an example is:" 0.0034639197401702404
time 13 is done ", an example is:" 0.0027065116446465254
time 14 is done ", an example is:" 0.0021147162187844515
time 15 is done ", an example is:" 0.0016523200320079923
time 16 is done ", an example is:" 0.0012910299701616168
time 17 is done ", an example is:" 0.001008738181553781
ti

time 145 is done ", an example is:" 1.9389710782645232e-17
time 146 is done ", an example is:" 1.5150027985867966e-17
time 147 is done ", an example is:" 1.1837379736468678e-17
time 148 is done ", an example is:" 9.249062943799413e-18
time 149 is done ", an example is:" 7.226697335974641e-18
time 150 is done ", an example is:" 5.646534211814592e-18
time 151 is done ", an example is:" 4.411884429517943e-18
time 152 is done ", an example is:" 3.4471980682226223e-18
time 153 is done ", an example is:" 2.693446692236233e-18
time 154 is done ", an example is:" 2.1045077445785846e-18
time 155 is done ", an example is:" 1.644343861092885e-18
time 156 is done ", an example is:" 1.284797710330253e-18
time 157 is done ", an example is:" 1.003868666141039e-18
time 158 is done ", an example is:" 7.843664678303189e-19
time 159 is done ", an example is:" 6.128598219005516e-19
time 160 is done ", an example is:" 4.788541845227e-19
time 161 is done ", an example is:" 3.741497401339202e-19
time 162 is 

In [19]:
diff_2 = (prediction_2-Z_2).tolist()

In [20]:
import csv

with open("prediction_22 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(prediction_2)

In [21]:
import csv

with open("diff_22 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(diff_2)

## $\Delta x=\frac{1}{80}$ $\Delta t=\frac{1}{40}$

In [22]:
x_3 = arange(0, 2 * np.pi, 1/80)
t_3 = arange(0, 2 * np.pi, 1/40)
X_3,T_3 = meshgrid(x_3, t_3) # grid of point
Z_3 = heat_equ_analytical_solu(X_3, T_3) # evaluation of the function on the grid

In [23]:
padding_3 = []
for z in Z_3:
    p = np.hstack(([0,0,0], z, [0,0,0]))
    padding_3.append(p.tolist())

In [24]:
model_3 = ResNet(7, 6, 1)
optimizer_3 = optim.Adam(model_3.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion_3 = nn.MSELoss()
model_3.train()

ResNet(
  (linear1): Linear(in_features=7, out_features=6, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (relu2): ReLU()
)

In [25]:
model_3.load_model("model_xx delta x=1 80 delta t=1 40")

In [26]:
prediction_3 = []
prediction_3.append(torch.FloatTensor(Z_3[0]).tolist())
for i in range(len(Z_3)-1):
    prediction_3.append(calc_next_time(Z_3[i], model_3))
    print("time", i, "is done", '", an example is:"', prediction_3[i+1][0])

time 0 is done ", an example is:" 0.03360804542899132
time 1 is done ", an example is:" 0.02625943534076214
time 2 is done ", an example is:" 0.02051764354109764
time 3 is done ", an example is:" 0.016031334176659584
time 4 is done ", an example is:" 0.012525982223451138
time 5 is done ", an example is:" 0.009787097573280334
time 6 is done ", an example is:" 0.00764708686619997
time 7 is done ", an example is:" 0.005975003819912672
time 8 is done ", an example is:" 0.004668531473726034
time 9 is done ", an example is:" 0.0036477278918027878
time 10 is done ", an example is:" 0.002850129036232829
time 11 is done ", an example is:" 0.0022269305773079395
time 12 is done ", an example is:" 0.00173999834805727
time 13 is done ", an example is:" 0.0013595366617664695
time 14 is done ", an example is:" 0.0010622654808685184
time 15 is done ", an example is:" 0.0008299945620819926
time 16 is done ", an example is:" 0.0006485110498033464
time 17 is done ", an example is:" 0.0005067099700681865


time 145 is done ", an example is:" 9.739852362284276e-18
time 146 is done ", an example is:" 7.610172477689937e-18
time 147 is done ", an example is:" 5.946160122786919e-18
time 148 is done ", an example is:" 4.645994740843926e-18
time 149 is done ", an example is:" 3.6301189277262206e-18
time 150 is done ", an example is:" 2.8363706807859017e-18
time 151 is done ", an example is:" 2.2161804359956935e-18
time 152 is done ", an example is:" 1.7315986978153115e-18
time 153 is done ", an example is:" 1.352973833019297e-18
time 154 is done ", an example is:" 1.0571375466258054e-18
time 155 is done ", an example is:" 8.259878181097262e-19
time 156 is done ", an example is:" 6.453803759842856e-19
time 157 is done ", an example is:" 5.042639321694102e-19
time 158 is done ", an example is:" 3.940034965518707e-19
time 159 is done ", an example is:" 3.078521704653783e-19
time 160 is done ", an example is:" 2.405383577155267e-19
time 161 is done ", an example is:" 1.8794312536680522e-19
time 162

In [27]:
diff_3 = (prediction_3-Z_3).tolist()

In [28]:
import csv

with open("prediction_32 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(prediction_3)

In [29]:
import csv

with open("diff_32 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(diff_3)

## $\Delta x=\frac{1}{160}$ $\Delta t=\frac{1}{40}$

In [30]:
x_4 = arange(0, 2 * np.pi, 1/160)
t_4 = arange(0, 2 * np.pi, 1/40)
X_4,T_4 = meshgrid(x_4, t_4) # grid of point
Z_4 = heat_equ_analytical_solu(X_4, T_4) # evaluation of the function on the grid

In [31]:
padding_4 = []
for z in Z_4:
    p = np.hstack(([0,0,0], z, [0,0,0]))
    padding_4.append(p.tolist())

In [32]:
model_4 = ResNet(7, 6, 1)
optimizer_4 = optim.Adam(model_4.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion_4 = nn.MSELoss()
model_4.train()

ResNet(
  (linear1): Linear(in_features=7, out_features=6, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (relu2): ReLU()
)

In [33]:
model_4.load_model("model_xx delta x=1 160 delta t=1 40")

In [34]:
prediction_4 = []
prediction_4.append(torch.FloatTensor(Z_4[0]).tolist())
for i in range(len(Z_4)-1):
    prediction_4.append(calc_next_time(Z_4[i], model_4))
    print("time", i, "is done", '", an example is:"', prediction_4[i+1][0])

time 0 is done ", an example is:" 0.01682347245514393
time 1 is done ", an example is:" 0.01314491592347622
time 2 is done ", an example is:" 0.010270697996020317
time 3 is done ", an example is:" 0.008024944923818111
time 4 is done ", an example is:" 0.006270240526646376
time 5 is done ", an example is:" 0.004899212624877691
time 6 is done ", an example is:" 0.0038279693108052015
time 7 is done ", an example is:" 0.0029909596778452396
time 8 is done ", an example is:" 0.0023369677364826202
time 9 is done ", an example is:" 0.0018259750213474035
time 10 is done ", an example is:" 0.0014267141232267022
time 11 is done ", an example is:" 0.0011147542390972376
time 12 is done ", an example is:" 0.0008710061083547771
time 13 is done ", an example is:" 0.000680555182043463
time 14 is done ", an example is:" 0.0005317475297488272
time 15 is done ", an example is:" 0.0004154776397626847
time 16 is done ", an example is:" 0.00032463084789924324
time 17 is done ", an example is:" 0.000253648264

time 143 is done ", an example is:" 7.986205475434093e-18
time 144 is done ", an example is:" 6.2399717812336104e-18
time 145 is done ", an example is:" 4.875563417016687e-18
time 146 is done ", an example is:" 3.809490562016507e-18
time 147 is done ", an example is:" 2.9765215463319863e-18
time 148 is done ", an example is:" 2.3256863277982198e-18
time 149 is done ", an example is:" 1.817160502618995e-18
time 150 is done ", an example is:" 1.4198268803185624e-18
time 151 is done ", an example is:" 1.1093729683327634e-18
time 152 is done ", an example is:" 8.668016333161286e-19
time 153 is done ", an example is:" 6.772699463570065e-19
time 154 is done ", an example is:" 5.291806284722505e-19
time 155 is done ", an example is:" 4.1347196775013047e-19
time 156 is done ", an example is:" 3.2306371173749877e-19
time 157 is done ", an example is:" 2.524238315939656e-19
time 158 is done ", an example is:" 1.972297787063615e-19
time 159 is done ", an example is:" 1.5410425218181483e-19
time 1

In [35]:
diff_4 = (prediction_4-Z_4).tolist()

In [36]:
import csv

with open("prediction_42 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(prediction_4)

In [37]:
import csv

with open("diff_42 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(diff_4)